In [26]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

In [27]:
base_path = r"C:\Users\pc\Documents\bhc project"


In [31]:
print("\n--- PHASE 1: LIFESTYLE DATA ---")
lifestyle_path = os.path.join(base_path, "lifestyle")




--- PHASE 1: LIFESTYLE DATA ---


In [32]:
path_pa = os.path.join(lifestyle_path, "Physical Activity.xpt")
df_pa = pd.read_sas(path_pa)
df_pa_clean = df_pa[['SEQN', 'PAD680']].rename(columns={'PAD680': 'Sedentary_Minutes'})
df_pa_clean.loc[df_pa_clean['Sedentary_Minutes'] >= 7777, 'Sedentary_Minutes'] = np.nan
print(f"✓ Physical Activity: {len(df_pa_clean)} records")


✓ Physical Activity: 8153 records


In [33]:
path_sleep = os.path.join(lifestyle_path, "Sleeping -SLQ_L.xpt")
df_sleep = pd.read_sas(path_sleep)
df_sleep_clean = df_sleep[['SEQN', 'SLD012']].rename(columns={'SLD012': 'Sleep_Hours'})
df_sleep_clean.loc[(df_sleep_clean['Sleep_Hours'] > 24) | (df_sleep_clean['Sleep_Hours'] < 0), 'Sleep_Hours'] = np.nan
print(f"✓ Sleep: {len(df_sleep_clean)} records")

✓ Sleep: 8501 records


In [34]:
path_smoke = os.path.join(lifestyle_path, "Smoking.xpt")
df_smoke = pd.read_sas(path_smoke)
df_smoke_clean = df_smoke[['SEQN', 'SMQ020']].rename(columns={'SMQ020': 'Smoker_History'})
df_smoke_clean.loc[~df_smoke_clean['Smoker_History'].isin([1, 2]), 'Smoker_History'] = np.nan
print(f"✓ Smoking: {len(df_smoke_clean)} records")

✓ Smoking: 9015 records


In [35]:
df_lifestyle = pd.merge(df_pa_clean, df_sleep_clean, on='SEQN', how='outer')
df_lifestyle = pd.merge(df_lifestyle, df_smoke_clean, on='SEQN', how='outer')
print(f"✓ Lifestyle merged: {df_lifestyle.shape}")

✓ Lifestyle merged: (9085, 4)


In [37]:
print("\n--- PHASE 2: NUTRITION DATA ---")
nutrition_path = os.path.join(base_path, "Nutrition")




--- PHASE 2: NUTRITION DATA ---


In [38]:
path_nut = os.path.join(nutrition_path, "DR1TOT_L sugar.xpt")
df_nut = pd.read_sas(path_nut)
df_diet = df_nut[['SEQN', 'DR1TSUGR', 'DR1TFIBE']].rename(columns={
    'DR1TSUGR': 'Sugar_Gm',
    'DR1TFIBE': 'Fiber_Gm'
})
df_diet.loc[df_diet['Sugar_Gm'] < 0, 'Sugar_Gm'] = np.nan
df_diet.loc[df_diet['Fiber_Gm'] < 0, 'Fiber_Gm'] = np.nan
print(f"✓ Diet: {len(df_diet)} records")

✓ Diet: 8860 records


In [39]:
path_alc = os.path.join(nutrition_path, "ALQ_L.xpt")
df_alc = pd.read_sas(path_alc)
df_alcohol_clean = df_alc[['SEQN', 'ALQ130']].rename(columns={'ALQ130': 'Alcohol_Drinks'})
df_alcohol_clean.loc[df_alcohol_clean['Alcohol_Drinks'] >= 777, 'Alcohol_Drinks'] = np.nan
df_alcohol_clean.loc[df_alcohol_clean['Alcohol_Drinks'] < 0.001, 'Alcohol_Drinks'] = 0
print(f"✓ Alcohol: {len(df_alcohol_clean)} records")

✓ Alcohol: 6337 records


In [40]:
df_nutrition = pd.merge(df_diet, df_alcohol_clean, on='SEQN', how='outer')
print(f"✓ Nutrition merged: {df_nutrition.shape}")

✓ Nutrition merged: (8860, 4)


In [41]:
print("\n--- PHASE 3: OMICS DATA ---")

omics_path = os.path.join(base_path, "Omics")


--- PHASE 3: OMICS DATA ---


In [42]:
path_bmi = os.path.join(omics_path, "BMX_L (1) (body measure).xpt")
df_bmi = pd.read_sas(path_bmi)
df_bmi_clean = df_bmi[['SEQN', 'BMXBMI']].rename(columns={'BMXBMI': 'BMI'})
df_bmi_clean.loc[(df_bmi_clean['BMI'] < 10) | (df_bmi_clean['BMI'] > 80), 'BMI'] = np.nan
print(f"✓ BMI: {len(df_bmi_clean)} records")

✓ BMI: 8860 records


In [43]:
path_ghb = os.path.join(omics_path, "GHB_L (hb1ac.xpt")
df_ghb = pd.read_sas(path_ghb)
df_ghb_clean = df_ghb[['SEQN', 'LBXGH']].rename(columns={'LBXGH': 'HbA1c'})
df_ghb_clean.loc[(df_ghb_clean['HbA1c'] < 3) | (df_ghb_clean['HbA1c'] > 20), 'HbA1c'] = np.nan
print(f"✓ HbA1c: {len(df_ghb_clean)} records")

✓ HbA1c: 7199 records


In [44]:
path_bp = os.path.join(omics_path, "Blood pressure and cholesterol.xpt")
df_bp = pd.read_sas(path_bp)
df_bp_clean = df_bp[['SEQN', 'BPQ020']].rename(columns={'BPQ020': 'Hypertension'})
df_bp_clean['Hypertension'] = df_bp_clean['Hypertension'].replace({1: 1, 2: 0})
df_bp_clean.loc[~df_bp_clean['Hypertension'].isin([0, 1]), 'Hypertension'] = np.nan
print(f"✓ Hypertension: {len(df_bp_clean)} records")


✓ Hypertension: 8501 records


In [45]:
path_vid = os.path.join(omics_path, "VID_L.xpt")
df_vid = pd.read_sas(path_vid)
df_vid_clean = df_vid[['SEQN', 'LBXVIDMS']].rename(columns={'LBXVIDMS': 'VitaminD_nmol'})
df_vid_clean.loc[(df_vid_clean['VitaminD_nmol'] < 0) | (df_vid_clean['VitaminD_nmol'] > 500), 'VitaminD_nmol'] = np.nan
print(f"✓ Vitamin D: {len(df_vid_clean)} records")

✓ Vitamin D: 8727 records


In [46]:
df_omics = pd.merge(df_bmi_clean, df_ghb_clean, on='SEQN', how='outer')
df_omics = pd.merge(df_omics, df_bp_clean, on='SEQN', how='outer')
df_omics = pd.merge(df_omics, df_vid_clean, on='SEQN', how='outer')
print(f"✓ Omics merged: {df_omics.shape}")


✓ Omics merged: (10746, 5)


In [47]:
print("\n--- PHASE 4: DEMOGRAPHICS ---")

file_demo = os.path.join(base_path, "DEMO_L.xpt")
df_demo = pd.read_sas(file_demo)
df_demo_clean = df_demo[['SEQN', 'RIDAGEYR', 'RIAGENDR']].rename(columns={
    'RIDAGEYR': 'Age',
    'RIAGENDR': 'Gender'
})

print(f"✓ Total participants: {len(df_demo_clean)}")



--- PHASE 4: DEMOGRAPHICS ---
✓ Total participants: 11933


In [48]:
df_adults = df_demo_clean[df_demo_clean['Age'] >= 20].copy()
print(f"✓ Adults (≥20 years): {len(df_adults)}")
print(f"✓ Excluded (children): {len(df_demo_clean) - len(df_adults)}")


✓ Adults (≥20 years): 7809
✓ Excluded (children): 4124


In [49]:
print("\n--- PHASE 5: DATA INTEGRATION ---")

df_final = df_adults.copy()
df_final = pd.merge(df_final, df_lifestyle, on='SEQN', how='left')
df_final = pd.merge(df_final, df_nutrition, on='SEQN', how='left')
df_final = pd.merge(df_final, df_omics, on='SEQN', how='left')

print(f"✓ Master dataset: {df_final.shape}")
print(f"✓ Columns: {list(df_final.columns)}")



--- PHASE 5: DATA INTEGRATION ---
✓ Master dataset: (7809, 13)
✓ Columns: ['SEQN', 'Age', 'Gender', 'Sedentary_Minutes', 'Sleep_Hours', 'Smoker_History', 'Sugar_Gm', 'Fiber_Gm', 'Alcohol_Drinks', 'BMI', 'HbA1c', 'Hypertension', 'VitaminD_nmol']


In [50]:
print("\n--- PHASE 6: BHC SCORING ---")

df_scored = df_final.copy()



--- PHASE 6: BHC SCORING ---


In [51]:
negative_vars = ['Sedentary_Minutes', 'Sugar_Gm', 'Alcohol_Drinks', 'BMI', 'HbA1c']

In [52]:
positive_vars = ['Fiber_Gm', 'VitaminD_nmol', 'Sleep_Hours']


In [53]:
scaler = MinMaxScaler(feature_range=(0, 100))


In [55]:
for var in negative_vars:
    score_col = f"Score_{var}"
    if var in df_scored.columns:
        values = df_scored[[var]].copy()
        mask = values[var].notna()
        if mask.sum() > 0:
            scaled = np.full(len(values), np.nan)
            scaled[mask] = scaler.fit_transform(values[mask]).flatten()
            df_scored[score_col] = 100 - scaled
        else:
            df_scored[score_col] = np.nan

In [56]:
df_scored['Score_Smoker'] = df_scored['Smoker_History'].replace({1: 0, 2: 100})
df_scored['Score_Hypertension'] = df_scored['Hypertension'].replace({1: 0, 0: 100})

In [57]:
score_columns = [c for c in df_scored.columns if c.startswith('Score_')]
df_scored['BHC_Score'] = df_scored[score_columns].mean(axis=1)

print(f"✓ BHC Score calculated")
print(f"✓ Population mean: {df_scored['BHC_Score'].mean():.2f}/100")


✓ BHC Score calculated
✓ Population mean: 71.57/100


In [58]:
print("\n--- PHASE 7: QUALITY CONTROL ---")



--- PHASE 7: QUALITY CONTROL ---


In [59]:
df_scored['Data_Completeness'] = df_scored[score_columns].notna().sum(axis=1)

print(f"  Before filtering: {len(df_scored)} participants")


  Before filtering: 7809 participants


In [60]:
threshold = 5
df_final_valid = df_scored[df_scored['Data_Completeness'] >= threshold].copy()

print(f"  After filtering (≥{threshold} scores): {len(df_final_valid)} participants")
print(f"  Excluded: {len(df_scored) - len(df_final_valid)}")


  After filtering (≥5 scores): 5958 participants
  Excluded: 1851


In [61]:
print("\n--- SAVING DATA ---")

output_csv = "BHC_Final_Dataset_Corrected.csv"
save_path_csv = os.path.join(base_path, output_csv)
df_final_valid.to_csv(save_path_csv, index=False)
print(f"✓ Saved: {output_csv}")



--- SAVING DATA ---
✓ Saved: BHC_Final_Dataset_Corrected.csv


In [63]:
print("\n" + "="*60)
print("DEMOGRAPHIC SUMMARY")
print("="*60)

n = len(df_final_valid)
print(f"Total Participants (N): {n}")

mean_age = df_final_valid['Age'].mean()
std_age = df_final_valid['Age'].std()
print(f"Age: {mean_age:.1f} ± {std_age:.1f} years")

num_females = len(df_final_valid[df_final_valid['Gender'] == 2])
perc_female = (num_females / n) * 100
print(f"Gender: {perc_female:.1f}% Female (n={num_females})")

mean_score = df_final_valid['BHC_Score'].mean()
std_score = df_final_valid['BHC_Score'].std()
print(f"BHC Score: {mean_score:.1f} ± {std_score:.1f}")

print("="*60)


DEMOGRAPHIC SUMMARY
Total Participants (N): 5958
Age: 53.9 ± 17.1 years
Gender: 55.1% Female (n=3284)
BHC Score: 73.4 ± 13.2


In [64]:
print("\n--- ANALYSIS 1: CORRELATION ANALYSIS ---")

analysis_vars = ['BHC_Score']
for var in ['BMI', 'HbA1c', 'Hypertension']:
    if var in df_final_valid.columns:
        analysis_vars.append(var)

print("\nPearson Correlation:")
pearson_corr = df_final_valid[analysis_vars].corr(method='pearson')
print(pearson_corr[['BHC_Score']].sort_values(by='BHC_Score', ascending=False))

print("\nSpearman Correlation:")
spearman_corr = df_final_valid[analysis_vars].corr(method='spearman')
print(spearman_corr[['BHC_Score']].sort_values(by='BHC_Score', ascending=False))



--- ANALYSIS 1: CORRELATION ANALYSIS ---

Pearson Correlation:
              BHC_Score
BHC_Score      1.000000
BMI           -0.304034
HbA1c         -0.325117
Hypertension  -0.713291

Spearman Correlation:
              BHC_Score
BHC_Score      1.000000
BMI           -0.340894
HbA1c         -0.357146
Hypertension  -0.697113


In [65]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(pearson_corr[['BHC_Score']].sort_values(by='BHC_Score', ascending=False),
            annot=True, cmap='RdBu_r', vmin=-1, vmax=1, cbar=True, ax=axes[0])
axes[0].set_title("Pearson Correlation with BHC Score")

sns.heatmap(spearman_corr[['BHC_Score']].sort_values(by='BHC_Score', ascending=False),
            annot=True, cmap='RdBu_r', vmin=-1, vmax=1, cbar=True, ax=axes[1])
axes[1].set_title("Spearman Correlation with BHC Score")

plt.tight_layout()
plt.savefig(os.path.join(base_path, 'correlation_analysis.png'), dpi=300, bbox_inches='tight')
print("\n✓ Saved: correlation_analysis.png")
plt.close()



✓ Saved: correlation_analysis.png


In [72]:
# ============================================================================
# LOGISTIC REGRESSION (COPY THIS ENTIRE BLOCK)
# ============================================================================
print("\n--- ANALYSIS 2: LOGISTIC REGRESSION ---")

# Prepare data for regression
df_reg = df_final_valid[['BHC_Score', 'Hypertension']].dropna()
df_reg = df_reg[df_reg['Hypertension'].isin([0, 1])]

print(f"Regression sample: {len(df_reg)} participants")

X = sm.add_constant(df_reg['BHC_Score'])
y = df_reg['Hypertension']

# Fit model
try:
    logit_model = sm.Logit(y, X).fit(disp=0)
    print("\nLogistic Regression Results:")
    print(logit_model.summary())
    
    # Calculate odds ratios
    odds_ratios = np.exp(logit_model.params)
    conf_int = np.exp(logit_model.conf_int())
    
    print("\nOdds Ratios (95% CI):")
    print(f"BHC_Score: {odds_ratios['BHC_Score']:.3f} [{conf_int.loc['BHC_Score', 0]:.3f} - {conf_int.loc['BHC_Score', 1]:.3f}]")
    print("\nInterpretation: For each 1-point increase in BHC Score,")
    print(f"the odds of hypertension decrease by {(1-odds_ratios['BHC_Score'])*100:.1f}%")
except Exception as e:
    print(f"Regression failed: {e}")


--- ANALYSIS 2: LOGISTIC REGRESSION ---
Regression sample: 5954 participants

Logistic Regression Results:
                           Logit Regression Results                           
Dep. Variable:           Hypertension   No. Observations:                 5954
Model:                          Logit   Df Residuals:                     5952
Method:                           MLE   Df Model:                            1
Date:                Thu, 05 Feb 2026   Pseudo R-squ.:                  0.4890
Time:                        10:08:05   Log-Likelihood:                -2023.0
converged:                       True   LL-Null:                       -3958.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         14.5318      0.401     36.225      0.000      13.746      15.318
BHC_Score     -0.2100  

In [73]:
print("\n--- ANALYSIS 3: SUBGROUP ANALYSIS ---")

# Create categorical variables
df_final_valid['Age_Group'] = pd.cut(df_final_valid['Age'],
                                      bins=[20, 40, 60, 100],
                                      labels=['Young (20-39)', 'Middle (40-59)', 'Senior (60+)'])

df_final_valid['BMI_Cat'] = pd.cut(df_final_valid['BMI'],
                                    bins=[0, 18.5, 25, 30, 100],
                                    labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

print("\nMean BHC Score by Gender:")
gender_scores = df_final_valid.groupby('Gender', observed=True)['BHC_Score'].agg(['mean', 'std', 'count'])
gender_scores.index = ['Male', 'Female']
print(gender_scores)

print("\nMean BHC Score by Age Group:")
age_scores = df_final_valid.groupby('Age_Group', observed=True)['BHC_Score'].agg(['mean', 'std', 'count'])
print(age_scores)

print("\nMean BHC Score by BMI Category:")
bmi_scores = df_final_valid.groupby('BMI_Cat', observed=True)['BHC_Score'].agg(['mean', 'std', 'count'])
print(bmi_scores)


--- ANALYSIS 3: SUBGROUP ANALYSIS ---

Mean BHC Score by Gender:
             mean        std  count
Male    71.688784  13.257527   2674
Female  74.854984  13.071657   3284

Mean BHC Score by Age Group:
                     mean        std  count
Age_Group                                  
Young (20-39)   79.551557  10.768384   1572
Middle (40-59)  73.127851  13.041555   1767
Senior (60+)    69.699692  13.375320   2569

Mean BHC Score by BMI Category:
                  mean        std  count
BMI_Cat                                 
Underweight  79.441937  12.876512     86
Normal       78.404305  11.894703   1531
Overweight   74.526752  12.762991   1893
Obese        69.338045  13.020688   2399


In [74]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(data=df_final_valid, x='Age_Group', y='BHC_Score', 
            hue='Age_Group', palette="Blues", ax=axes[0], legend=False)
axes[0].set_title("BHC Score by Age Group", fontsize=14, fontweight='bold')
axes[0].set_ylabel("BHC Health Score", fontsize=12)
axes[0].set_xlabel("Age Group", fontsize=12)

sns.boxplot(data=df_final_valid, x='BMI_Cat', y='BHC_Score',
            hue='BMI_Cat', palette="Reds_r", ax=axes[1], legend=False)
axes[1].set_title("BHC Score by BMI Category", fontsize=14, fontweight='bold')
axes[1].set_ylabel("BHC Health Score", fontsize=12)
axes[1].set_xlabel("BMI Category", fontsize=12)

plt.tight_layout()
plt.savefig(os.path.join(base_path, 'subgroup_analysis.png'), dpi=300, bbox_inches='tight')
print("\n✓ Saved: subgroup_analysis.png")
plt.close()


✓ Saved: subgroup_analysis.png


In [75]:
print("\n--- CREATING EXCEL REPORT ---")

output_excel = "BHC_Project_Report_Corrected.xlsx"
save_path_excel = os.path.join(base_path, output_excel)

# Create summary tables
summary_gender = df_final_valid.groupby('Gender', observed=True)[['Age', 'BHC_Score', 'BMI', 'HbA1c']].agg(['mean', 'std'])
summary_gender.index = ['Male', 'Female']

summary_age = df_final_valid.groupby('Age_Group', observed=True)[['BHC_Score', 'BMI', 'HbA1c']].agg(['mean', 'std'])

summary_bmi = df_final_valid.groupby('BMI_Cat', observed=True)[['BHC_Score', 'Age', 'HbA1c']].agg(['mean', 'std'])


--- CREATING EXCEL REPORT ---


In [76]:
top_10 = df_final_valid.nlargest(10, 'BHC_Score')[['SEQN', 'Age', 'Gender', 'BHC_Score', 'BMI', 'HbA1c', 'Hypertension']]
bottom_10 = df_final_valid.nsmallest(10, 'BHC_Score')[['SEQN', 'Age', 'Gender', 'BHC_Score', 'BMI', 'HbA1c', 'Hypertension']]


In [77]:
with pd.ExcelWriter(save_path_excel, engine='openpyxl') as writer:
    summary_gender.to_excel(writer, sheet_name='Summary_by_Gender')
    summary_age.to_excel(writer, sheet_name='Summary_by_Age')
    summary_bmi.to_excel(writer, sheet_name='Summary_by_BMI')
    top_10.to_excel(writer, sheet_name='Top_10_Healthiest', index=False)
    bottom_10.to_excel(writer, sheet_name='Bottom_10_Healthiest', index=False)
    df_final_valid.to_excel(writer, sheet_name='Full_Dataset', index=False)
    pearson_corr.to_excel(writer, sheet_name='Correlations')

print(f"✓ Saved: {output_excel}")

✓ Saved: BHC_Project_Report_Corrected.xlsx


In [78]:
print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)
print("\nFiles created:")
print(f"  1. {output_csv}")
print(f"  2. {output_excel}")
print(f"  3. correlation_analysis.png")
print(f"  4. subgroup_analysis.png")
print("\nKey Findings:")
print(f"  • {n} adult participants analyzed")
print(f"  • Mean BHC Score: {mean_score:.1f} ± {std_score:.1f}")
print(f"  • Strong negative correlation with hypertension")
print(f"  • Score declines with age and BMI")
print("="*60)




ANALYSIS COMPLETE!

Files created:
  1. BHC_Final_Dataset_Corrected.csv
  2. BHC_Project_Report_Corrected.xlsx
  3. correlation_analysis.png
  4. subgroup_analysis.png

Key Findings:
  • 5958 adult participants analyzed
  • Mean BHC Score: 73.4 ± 13.2
  • Strong negative correlation with hypertension
  • Score declines with age and BMI
